# Using RegVelo to predict perturbation

## Library imports

In [1]:
import numpy as np
import pandas as pd

import scanpy as sc

from rgv_tools import DATA_DIR
from rgv_tools.perturbation import (
    get_list_name,
    Multiple_TFScanning,
    split_elements,
    TFScanning,
)

/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packages/anndata/utils.py:429: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/home/icb/weixu.wang/miniconda3/envs/dynamo/lib/python3.10/site-packag

## Constants

In [2]:
DATASET = "zebrafish"

In [3]:
SAVE_DATA = True
if SAVE_DATA:
    (DATA_DIR / DATASET / "results").mkdir(parents=True, exist_ok=True)

In [4]:
TERMINAL_STATES = [
    "mNC_head_mesenchymal",
    "mNC_arch2",
    "mNC_hox34",
    "Pigment",
]

single_ko = ["elk3", "erf", "fli1a", "mitfa", "nr2f5", "rarga", "rxraa", "smarcc1a", "tfec", "nr2f2"]
multiple_ko = ["fli1a_elk3", "mitfa_tfec", "tfec_mitfa_bhlhe40", "fli1a_erf_erfl3", "erf_erfl3"]

## Data loading

In [5]:
adata = sc.read_h5ad(DATA_DIR / DATASET / "processed" / "adata_preprocessed.h5ad")

## Perturbation prediction

### Single gene knockout

In [6]:
single_ko = set(single_ko).intersection(adata.var_names)
single_ko = list(single_ko)

In [7]:
for nrun in range(3):
    model_name = "rgv_model_" + str(nrun)
    coef_name = "coef_single_regvelo_" + str(nrun)
    pval_name = "pval_single_regvelo_" + str(nrun)

    model = DATA_DIR / DATASET / "processed" / "perturb_repeat_runs" / model_name
    coef_save = DATA_DIR / DATASET / "results" / coef_name
    pval_save = DATA_DIR / DATASET / "results" / pval_name

    ## Perturbation
    d = TFScanning(model, adata, 8, "cell_type", TERMINAL_STATES, single_ko, 0)

    coef = pd.DataFrame(np.array(d["coefficient"]))
    coef.index = d["TF"]
    coef.columns = get_list_name(d["coefficient"][0])
    pval = pd.DataFrame(np.array(d["pvalue"]))
    pval.index = d["TF"]
    pval.columns = get_list_name(d["pvalue"][0])
    coef = coef.loc[single_ko, :]

    coef.to_csv(coef_save)
    pval.to_csv(pval_save)

INFO     File                                                                                                      
         /ictstr01/home/icb/weixu.wang/regulatory_velo/data/zebrafish/processed/perturb_repeat_runs/rgv_model_0/mod
         el.pt already downloaded                                                                                  


  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 17:45:31,171 - INFO - Using pre-computed Schur decomposition


Defaulting to `'gmres'` solver.


...

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 17:50:43,044 - INFO - Using pre-computed Schur decomposition


  0%|          | 0/4 [00:00<?, ?/s]

Done erf


### Multiple knockout

In [8]:
multiple_ko_list = split_elements(multiple_ko)

In [9]:
for nrun in [0, 1, 2]:
    model_name = "rgv_model_" + str(nrun)
    coef_name = "coef_multiple_regvelo_" + str(nrun)
    pval_name = "pval_multiple_regvelo_" + str(nrun)

    model = DATA_DIR / DATASET / "processed" / "perturb_repeat_runs" / model_name
    coef_save = DATA_DIR / DATASET / "results" / coef_name
    pval_save = DATA_DIR / DATASET / "results" / pval_name

    ## Perturbatiom
    d = Multiple_TFScanning(model, adata, 8, "cell_type", TERMINAL_STATES, multiple_ko_list, 0)
    coef = pd.DataFrame(np.array(d["coefficient"]))
    coef.index = d["TF"]
    coef.columns = get_list_name(d["coefficient"][0])
    pval = pd.DataFrame(np.array(d["pvalue"]))
    pval.index = d["TF"]
    pval.columns = get_list_name(d["pvalue"][0])

    coef.to_csv(coef_save)
    pval.to_csv(pval_save)

INFO     File                                                                                                      
         /ictstr01/home/icb/weixu.wang/regulatory_velo/data/zebrafish/processed/perturb_repeat_runs/rgv_model_0/mod
         el.pt already downloaded                                                                                  


  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 17:50:54,421 - INFO - Using pre-computed Schur decomposition


  0%|          | 0/4 [00:00<?, ?/s]

...

  0%|          | 0/697 [00:00<?, ?cell/s]

  0%|          | 0/697 [00:00<?, ?cell/s]

2024-11-24 17:54:01,592 - INFO - Using pre-computed Schur decomposition


  0%|          | 0/4 [00:00<?, ?/s]

Done erf_erfl3
